In [1]:
import pandas as pd
from pathlib import Path
import os

def calculate(filename):
    directory = os.path.join(Path().absolute(), "speed_prediction_logs")
    path = os.path.join(directory, filename + ".csv")
    df = pd.read_csv(path)

    df = df.fillna(0)

    #df = df[df["avgSpeedPerTest"] > 0]
    #df = df[df["avgIntratecla"] > 0]
    #df = df[df["avgScrollSpeed"] > 0]
    
    df["sum" + filename] = df["avgSpeedPerTest"] + df["avgIntratecla"] + df["avgScrollSpeed"]
    df["mul" + filename] = df["avgSpeedPerTest"] * df["avgIntratecla"] * df["avgScrollSpeed"]

    column_names = df.columns.values.tolist()
    column_names.remove('name')
    column_names.remove("sum" + filename)
    column_names.remove("mul" + filename)
    for name in column_names:
        df.pop(name)
    
    return df

directory = os.path.join(Path().absolute(), "speed_prediction_logs")
path = os.path.join(directory, "eventAmountPerTest.csv")
df = pd.read_csv(path)

logs = ["100","200", "300","400" ,"500","600", "700","800","900" ,"1000", "all"]
for name in logs:
    df = df.merge(right=calculate(name), on='name', how='left')
    
speed_df = df.copy(deep=True)
#speed_df.to_csv('speed_prediction.csv')

for name in logs:
    df["sum"+name] = df["sum"+name]  / df["sumall"]
    df["mul"+name] = df["mul"+name]  / df["mulall"]

speed_df_norm = df
#speed_df_norm.to_csv('speed_prediction_normalized.csv')

In [ ]:
speed_df

In [2]:
def remove_columns_that_starts_with(df, prefix):
    column_names = df.columns.values.tolist()
    for name in column_names:
        if name.startswith(prefix):
            df.pop(name)
    return df

speed_df_only_mul = speed_df.copy(deep=True)
speed_df_only_mul = remove_columns_that_starts_with(speed_df_only_mul, "mul")
speed_df_only_mul

,count,name,sum100,sum200,sum300,sum400,sum500,sum600,sum700,sum800,sum900,sum1000,sumall
0,709,dull_fireant_6vu766,0.045540,4859.049684,15263.060451,15263.063607,4666.493026,4666.492865,4666.498093,4666.500330,4666.500330,4666.500330,4666.500330
1,691,strong_snake_uwcskr,0.158129,0.163683,0.096186,0.086605,235.921983,235.928434,235.945395,235.945395,235.945395,235.945395,235.945395
2,2553,odd_bullfrog_4y9pil,231.115670,231.329048,231.581091,231.666468,231.739413,231.831053,231.860819,231.784742,232.264257,322.481931,364.675270
3,1588,kind_grasshopper_awambl,0.183479,148.524279,172.107323,234.404427,292.194093,329.311709,365.301993,378.700878,375.109762,402.979297,402.979987
4,711,moody_rabbit_53w523,0.153900,5620.116862,5620.120966,13648.615949,4134.819966,4134.821749,4134.818585,4134.819317,4134.819317,4134.819317,4134.819317
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,1228,unlucky_dodo_aljom1,0.058073,0.029834,0.024748,25963.020699,25963.023385,25963.028833,25963.036910,25963.040803,25963.041097,19805.040174,19805.039393
221,338,lovely_lizard_2ibeyu,1154.708781,1681.282716,971.730473,887.295288,887.295288,887.295288,887.295288,887.295288,887.295288,887.295288,887.295288
222,532,chilly_falcon_kpe1cd,0.142477,496.765752,568.183939,770.901357,970.509681,970.517461,970.517461,970.517461,970.517461,970.517461,970.517461
223,1217,chilly_bullfrog_fzw9lp,0.076359,2165.489183,1640.038876,1274.617877,1419.750293,1588.354839,1544.962294,1403.251443,1359.990067,1398.081954,1392.891778


In [ ]:
speed_df_only_mul.describe()

In [ ]:
speed_df_only_mul_norm = speed_df_norm.copy(deep=True)
speed_df_only_mul_norm = remove_columns_that_starts_with(speed_df_only_mul_norm, "mul")
speed_df_only_mul_norm.describe()

In [3]:
df_test = speed_df_only_mul.copy(deep=True)
column_names = df_test.columns.values.tolist()
for name in column_names:
    if "all" in name:
        continue
    if name.startswith("sum"):
        amount = name.replace("sum","")
        amount = int(amount)
        df_test.loc[amount > df["count"], name] = None

        df_test[name] = df_test[name] / df_test["sumall"]

df_test

,count,name,sum100,sum200,sum300,sum400,sum500,sum600,sum700,sum800,sum900,sum1000,sumall
0,709,dull_fireant_6vu766,0.000010,1.041262,3.270772,3.270773,0.999998,0.999998,1.000000,NaN,NaN,NaN,4666.500330
1,691,strong_snake_uwcskr,0.000670,0.000694,0.000408,0.000367,0.999901,0.999928,NaN,NaN,NaN,NaN,235.945395
2,2553,odd_bullfrog_4y9pil,0.633757,0.634343,0.635034,0.635268,0.635468,0.635719,0.635801,0.635592,0.636907,0.884299,364.675270
3,1588,kind_grasshopper_awambl,0.000455,0.368565,0.427087,0.581678,0.725083,0.817191,0.906502,0.939751,0.930840,0.999998,402.979987
4,711,moody_rabbit_53w523,0.000037,1.359217,1.359218,3.300898,1.000000,1.000001,1.000000,NaN,NaN,NaN,4134.819317
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,1228,unlucky_dodo_aljom1,0.000003,0.000002,0.000001,1.310930,1.310930,1.310930,1.310931,1.310931,1.310931,1.000000,19805.039393
221,338,lovely_lizard_2ibeyu,1.301380,1.894840,1.095160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,887.295288
222,532,chilly_falcon_kpe1cd,0.000147,0.511857,0.585444,0.794320,0.999992,NaN,NaN,NaN,NaN,NaN,970.517461
223,1217,chilly_bullfrog_fzw9lp,0.000055,1.554672,1.177435,0.915088,1.019283,1.140329,1.109176,1.007438,0.976379,1.003726,1392.891778


In [4]:
df_test.describe()

,count,sum100,sum200,sum300,sum400,sum500,sum600,sum700,sum800,sum900,sum1000,sumall
count,225.000000,224.000000,214.000000,191.000000,166.000000,151.000000,136.000000,124.000000,109.000000,94.000000,83.000000,225.000000
mean,1082.235556,0.456382,0.701520,0.778238,0.881294,0.844149,0.907237,1.003157,0.994785,0.980420,0.959390,1391.692857
std,1034.921621,0.760156,0.975228,0.824701,0.890702,0.539270,0.546026,0.808110,0.694472,0.701895,0.589182,3026.132083
min,98.000000,0.000002,0.000001,0.000001,0.000001,0.000001,0.000001,0.000002,0.000002,0.000002,0.000002,0.049613
25%,390.000000,0.000333,0.000504,0.000667,0.451995,0.658524,0.787311,0.857853,0.865770,0.838523,0.840705,250.913668
50%,759.000000,0.001205,0.749060,0.877699,0.908763,0.999669,0.999965,0.999994,0.999998,0.999987,0.999975,412.903113
75%,1417.000000,0.856485,0.999997,1.006594,1.066644,1.055879,1.044181,1.051029,1.054260,1.052664,1.051399,725.104907
max,6085.000000,7.556039,7.556053,5.865020,7.417522,3.550170,3.778544,6.170066,5.198273,5.198329,3.550173,19805.039393
